This notebook is a continuation of the initial Heart notebook.
It is going to cover the deep learning model using tanh as the activation function in the output layer.

In [82]:
import io
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from sklearn.model_selection import GridSearchCV, train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [83]:
training_path = '/content/drive/MyDrive/Colab Notebooks/Ind_Proj/deeplearning1_training.csv'
traininglabel_path = '/content/drive/MyDrive/Colab Notebooks/Ind_Proj/deeplearning1_traininglabel.csv'
eval_path = '/content/drive/MyDrive/Colab Notebooks/Ind_Proj/deeplearning1_evaluate.csv'
eval_label_path = '/content/drive/MyDrive/Colab Notebooks/Ind_Proj/deeplearning1_evaluatelabel.csv'
test_path = '/content/drive/MyDrive/Colab Notebooks/Ind_Proj/deeplearning1_test.csv'
testlabels_path = '/content/drive/MyDrive/Colab Notebooks/Ind_Proj/deeplearning1_testlabels.csv'
main_data_path = '/content/drive/MyDrive/Colab Notebooks/Ind_Proj/deeplearning1.csv'

deeplearning1_training = pd.read_csv(training_path)
deeplearning1_traininglabel = pd.read_csv(traininglabel_path)
deeplearning1_evaluate = pd.read_csv(eval_path)
deeplearning1_evaluatelabel = pd.read_csv(eval_label_path)
deeplearning1_test = pd.read_csv(test_path)
deeplearning1_testlabels = pd.read_csv(testlabels_path)
main_data = pd.read_csv(main_data_path)

In [84]:
print("Training shape:", deeplearning1_training.shape)
print("Evaluation shape:", deeplearning1_evaluate.shape)
print("Test shape:", deeplearning1_test.shape)
print("Main data shape: ", main_data.shape)
#no need to reshape or normalize

Training shape: (700, 11)
Evaluation shape: (109, 11)
Test shape: (109, 11)
Main data shape:  (918, 12)


In [85]:
deeplearning1_evaluatelabel

,HeartDisease
0,Normal
1,Normal
2,Normal
3,Normal
4,Heart Disease
...,...
104,Heart Disease
105,Normal
106,Heart Disease
107,Normal


In [86]:
#1: Normal
#-1: Heart Disease
label_encoder = LabelEncoder()

deeplearning1_traininglabel = label_encoder.fit_transform(deeplearning1_traininglabel.iloc[:, 0])
deeplearning1_traininglabel = np.where(deeplearning1_traininglabel == 0, -1, 1)
deeplearning1_traininglabel

array([ 1, -1,  1, -1,  1,  1,  1,  1, -1,  1,  1, -1,  1, -1,  1,  1, -1,
        1, -1, -1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1, -1,  1, -1, -1,
        1,  1, -1,  1,  1,  1,  1, -1,  1,  1, -1,  1,  1,  1,  1, -1, -1,
       -1,  1,  1,  1,  1, -1, -1,  1, -1,  1,  1,  1, -1,  1,  1,  1,  1,
       -1,  1, -1,  1, -1,  1, -1,  1, -1,  1,  1, -1,  1,  1, -1,  1, -1,
       -1, -1,  1, -1,  1,  1,  1,  1, -1,  1, -1,  1,  1,  1,  1, -1,  1,
       -1, -1, -1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1, -1, -1, -1,  1,
       -1, -1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1,  1, -1,
        1,  1, -1, -1, -1, -1, -1,  1, -1,  1,  1,  1,  1, -1,  1,  1,  1,
        1,  1, -1, -1,  1, -1,  1, -1, -1,  1,  1,  1, -1, -1,  1,  1,  1,
        1,  1,  1,  1, -1, -1, -1,  1,  1,  1, -1,  1, -1,  1,  1, -1,  1,
       -1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,
        1,  1,  1, -1,  1, -1, -1, -1,  1,  1, -1,  1, -1,  1,  1,  1, -1,
       -1,  1,  1,  1, -1

In [87]:
deeplearning1_evaluatelabel = label_encoder.fit_transform(deeplearning1_evaluatelabel.iloc[:, 0])
deeplearning1_evaluatelabel = np.where(deeplearning1_evaluatelabel == 0, -1, 1)
deeplearning1_evaluatelabel

array([ 1,  1,  1,  1, -1,  1, -1, -1, -1, -1, -1,  1, -1,  1,  1,  1, -1,
        1, -1, -1, -1,  1, -1, -1,  1, -1,  1, -1,  1,  1,  1, -1, -1,  1,
       -1, -1, -1, -1,  1,  1,  1, -1,  1,  1, -1, -1, -1,  1, -1,  1,  1,
        1, -1,  1,  1, -1,  1, -1,  1, -1, -1, -1, -1, -1,  1,  1,  1,  1,
        1,  1,  1, -1,  1,  1, -1, -1, -1,  1, -1,  1,  1,  1,  1,  1, -1,
        1, -1, -1,  1,  1, -1, -1, -1, -1,  1,  1, -1, -1,  1,  1,  1, -1,
        1,  1, -1,  1, -1,  1, -1])

In [88]:
deeplearning1_testlabels = label_encoder.fit_transform(deeplearning1_testlabels.iloc[:, 0])
deeplearning1_testlabels = np.where(deeplearning1_testlabels == 0, -1, 1)
deeplearning1_testlabels

array([ 1,  1,  1,  1,  1, -1,  1, -1, -1, -1, -1,  1,  1,  1, -1,  1, -1,
        1,  1, -1,  1,  1,  1,  1,  1,  1, -1, -1,  1, -1,  1,  1, -1, -1,
        1,  1, -1, -1,  1, -1,  1, -1,  1, -1,  1,  1, -1,  1,  1, -1,  1,
       -1, -1,  1, -1, -1, -1,  1, -1,  1,  1,  1,  1, -1, -1,  1,  1, -1,
       -1,  1, -1,  1,  1,  1,  1, -1,  1,  1, -1, -1, -1,  1,  1,  1, -1,
        1, -1,  1, -1,  1, -1, -1, -1,  1,  1,  1, -1,  1, -1, -1, -1,  1,
       -1, -1, -1, -1, -1, -1,  1])

In [89]:
#split the main data part randomly into training, training label, evaluation, evaluation labels, test and test labels
#make sure that the training dataset has equal amounts of Normal and Heart Disease
#randomize the data
main_data = main_data.sample(frac=1, random_state=22)#.reset_index(drop=True)
training = main_data.iloc[:700]
remaining = main_data.iloc[700:]
evaluation = remaining.iloc[:109]
test = remaining.iloc[109:]
deeplearning1_training = training.iloc[:, :-1]
deeplearning1_traininglabel = training.iloc[:,-1]
deeplearning1_evaluate = evaluation.iloc[:, :-1]
deeplearning1_evaluatelabel = evaluation.iloc[:,-1]
deeplearning1_test = test.iloc[:, :-1]
deeplearning1_testlabels = test.iloc[:,-1]

In [90]:
#make sure the number of Normal = number of Heart Disease, might reduce the number of records for training.
#current number: 700
#seems like a nice proportion for both sides so i'll leave it like this for now.
normal_train = deeplearning1_traininglabel[deeplearning1_traininglabel=='Normal']
heart_train = deeplearning1_traininglabel[deeplearning1_traininglabel=='Heart Disease']

In [91]:
print(heart_train.shape)
print(normal_train.shape)

(383,)
(317,)


In [92]:
deeplearning1_evaluatelabel = label_encoder.fit_transform(deeplearning1_evaluatelabel.values)
deeplearning1_evaluatelabel = np.where(deeplearning1_evaluatelabel == 0, -1, 1)
deeplearning1_evaluatelabel

array([ 1,  1, -1, -1,  1,  1,  1,  1,  1, -1, -1,  1,  1, -1, -1, -1,  1,
        1,  1, -1, -1, -1, -1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1,
        1, -1,  1, -1,  1, -1, -1, -1, -1, -1,  1,  1, -1,  1,  1, -1,  1,
       -1, -1,  1,  1, -1, -1,  1, -1, -1,  1,  1, -1,  1, -1, -1,  1, -1,
       -1, -1,  1,  1,  1,  1, -1, -1,  1, -1, -1,  1, -1, -1, -1, -1, -1,
        1,  1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1,  1,  1, -1, -1,
       -1,  1,  1, -1,  1, -1, -1])

In [93]:
deeplearning1_traininglabel = label_encoder.fit_transform(deeplearning1_traininglabel.values)
deeplearning1_traininglabel = np.where(deeplearning1_traininglabel == 0, -1, 1)
deeplearning1_traininglabel

array([-1, -1, -1, -1, -1,  1,  1, -1,  1, -1,  1, -1,  1,  1, -1,  1,  1,
       -1,  1,  1,  1, -1,  1, -1, -1,  1, -1, -1,  1,  1,  1,  1, -1,  1,
       -1,  1, -1,  1,  1,  1, -1,  1, -1,  1, -1,  1, -1,  1, -1, -1, -1,
        1, -1, -1, -1,  1, -1, -1, -1, -1,  1,  1, -1,  1, -1, -1,  1,  1,
        1, -1, -1, -1, -1,  1,  1,  1, -1,  1, -1, -1,  1,  1, -1,  1, -1,
       -1, -1,  1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,
       -1, -1, -1,  1,  1, -1, -1, -1, -1,  1,  1, -1, -1, -1,  1,  1, -1,
        1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1,  1, -1,  1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1,  1, -1,  1,  1,  1, -1, -1,  1, -1, -1, -1,
       -1, -1,  1,  1,  1,  1, -1, -1,  1,  1,  1,  1, -1, -1,  1,  1, -1,
       -1, -1, -1, -1,  1,  1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1,  1,
       -1,  1, -1, -1,  1,  1, -1,  1, -1,  1,  1,  1, -1, -1, -1, -1, -1,
       -1, -1,  1, -1,  1,  1,  1,  1, -1,  1, -1,  1, -1, -1,  1,  1, -1,
        1, -1, -1,  1, -1

In [94]:
deeplearning1_testlabels = label_encoder.fit_transform(deeplearning1_testlabels.values)
deeplearning1_testlabels = np.where(deeplearning1_testlabels == 0, -1, 1)
deeplearning1_testlabels

array([ 1, -1,  1,  1, -1,  1,  1,  1, -1, -1,  1, -1,  1,  1, -1, -1, -1,
       -1,  1, -1,  1, -1, -1, -1, -1, -1,  1,  1,  1,  1, -1, -1, -1, -1,
        1, -1, -1, -1, -1,  1, -1,  1,  1, -1, -1, -1,  1,  1, -1, -1, -1,
        1,  1,  1, -1,  1,  1, -1,  1,  1,  1,  1, -1, -1, -1,  1, -1, -1,
       -1, -1,  1, -1, -1,  1,  1,  1, -1, -1,  1, -1,  1, -1, -1, -1,  1,
        1, -1, -1, -1, -1, -1,  1, -1, -1, -1,  1, -1,  1, -1, -1,  1, -1,
       -1, -1, -1, -1,  1, -1,  1])

In [95]:
model = keras.Sequential()

In [125]:
#add the hidden layers
#we'll start with two hidden layers, one with 512 neurons and the second as an output layer with 2 neurons
model.add(layers.Dense(700, activation="relu"))
model.add(layers.Dense(400, activation="relu"))
model.add(layers.Dense(1, activation="tanh"))

In [126]:
#compile the model
opt = keras.optimizers.RMSprop(learning_rate=0.001)
#opt = keras.optimizers.Adam(learning_rate=0.001)

In [127]:
model.compile(
    optimizer=opt,
    #loss="sparse_categorical_crossentropy",
    loss = "binary_crossentropy",
    metrics=["accuracy"]
)

In [128]:
model.load_weights('/content/drive/MyDrive/Colab Notebooks/sigmoid weights')

In [129]:
model.fit(deeplearning1_training, deeplearning1_traininglabel, epochs = 30, batch_size = 15)

Epoch 1/30
47/47 [==============================] - 3s 5ms/step - loss: -1.4542 - accuracy: 0.0000e+00
Epoch 2/30
47/47 [==============================] - 0s 5ms/step - loss: -1.4544 - accuracy: 0.0000e+00
Epoch 3/30
47/47 [==============================] - 0s 5ms/step - loss: -1.4544 - accuracy: 0.0000e+00
Epoch 4/30
47/47 [==============================] - 0s 5ms/step - loss: -1.4544 - accuracy: 0.0000e+00
Epoch 5/30
47/47 [==============================] - 0s 5ms/step - loss: -1.4544 - accuracy: 0.0000e+00
Epoch 6/30
47/47 [==============================] - 0s 6ms/step - loss: -1.4544 - accuracy: 0.0000e+00
Epoch 7/30
47/47 [==============================] - 0s 5ms/step - loss: -1.4544 - accuracy: 0.0000e+00
Epoch 8/30
47/47 [==============================] - 0s 5ms/step - loss: -1.4544 - accuracy: 0.0000e+00
Epoch 9/30
47/47 [==============================] - 0s 5ms/step - loss: -1.4544 - accuracy: 0.0000e+00
Epoch 10/30
47/47 [==============================] - 0s 5ms/step - loss: 

In [102]:
#save the weights
#save & load the weights
model.save_weights('/content/drive/MyDrive/Colab Notebooks/tanhweights')

In [113]:
model.load_weights('/content/drive/MyDrive/Colab Notebooks/tanhweights')

In [124]:
#Evaluate model
test_loss, test_acc = model.evaluate(deeplearning1_evaluate, deeplearning1_evaluatelabel)

4/4 [==============================] - 0s 4ms/step - loss: -1.8397 - accuracy: 0.0000e+00


In [22]:
print(f"Evaluation Loss: {test_loss:.3f}")
print(f"Evaluation accuracy: {test_acc:.3f}")

Evaluation Loss: 1.686
Evaluation accuracy: 0.862


In [23]:
#Predictions
test = deeplearning1_test[0:10]
predictions = model.predict(test)

1/1 [==============================] - 0s 83ms/step


In [26]:
print(predictions)

[[ 1.]
 [-1.]
 [-1.]
 [ 1.]
 [-1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [-1.]
 [-1.]]


In [24]:
#predict first 10 patient results
print("1: Heart Disease")
print("-1: Normal")
print("-----------------------------------------------")
for x in range(10):
  print("                    ")
  print("Patient",x, "actual result: ", deeplearning1_testlabels[x])
  for y in range(predictions.shape[1]):
    print("Patient predicted result: ", y, ":", predictions[x, y])


0: Heart Disease
1: Normal
-----------------------------------------------
                    
Patient 0 actual result:  1
Patient predicted result:  0 : 1.0
                    
Patient 1 actual result:  0
Patient predicted result:  0 : -1.0
                    
Patient 2 actual result:  1
Patient predicted result:  0 : -1.0
                    
Patient 3 actual result:  1
Patient predicted result:  0 : 1.0
                    
Patient 4 actual result:  0
Patient predicted result:  0 : -1.0
                    
Patient 5 actual result:  1
Patient predicted result:  0 : 1.0
                    
Patient 6 actual result:  1
Patient predicted result:  0 : 1.0
                    
Patient 7 actual result:  1
Patient predicted result:  0 : 1.0
                    
Patient 8 actual result:  0
Patient predicted result:  0 : -1.0
                    
Patient 9 actual result:  0
Patient predicted result:  0 : -1.0


In [25]:
#Test accuracy
Test = deeplearning1_test
prediction = model.predict(Test)
#prediction = np.argmax(prediction, axis=1)


4/4 [==============================] - 0s 2ms/step


In [31]:
# if the probability is more than 0.5 (1) means it's normal
# and if it is less than 0.5 (0) means it's heart disease

prediction = prediction > 0

In [32]:
#classification report
print(classification_report(deeplearning1_testlabels, prediction))

              precision    recall  f1-score   support

           0       0.84      0.83      0.83        64
           1       0.76      0.78      0.77        45

    accuracy                           0.81       109
   macro avg       0.80      0.80      0.80       109
weighted avg       0.81      0.81      0.81       109



In [29]:
accuracy = accuracy_score(deeplearning1_testlabels, prediction)
print("Deep learning Accuracy:", accuracy)

Deep learning Accuracy: 0.8073394495412844


In [30]:
print("0: Heart Disease")
print("1: Normal")
print("-----------------------------------------------")
confuse_matrix2 = confusion_matrix(deeplearning1_testlabels, prediction)
print("Confusion Matrix:")
print(confuse_matrix2)
print("---------------------------------------------")
comp_table2 = pd.crosstab(deeplearning1_testlabels, prediction.flatten(), rownames=['Actual'], colnames=['Predicted'])
print(comp_table2)

0: Heart Disease
1: Normal
-----------------------------------------------
Confusion Matrix:
[[53 11]
 [10 35]]
---------------------------------------------
Predicted  False  True 
Actual                 
0             53     11
1             10     35
